<a href="https://colab.research.google.com/github/from-iqwerty-import-IQ/Deep_Learning_Projects/blob/main/mask_violation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50

In [9]:
training_dir = '/content/drive/Othercomputers/My Computer/complete dataset/train'
validate_dir = '/content/drive/Othercomputers/My Computer/complete dataset/validate'

In [13]:
training_with_mask_dir = os.path.join(training_dir,'with mask')
training_without_mask_dir = os.path.join(training_dir,'without mask')
training_improper_mask_dir = os.path.join(training_dir,'improper mask')

test_with_mask_dir = os.path.join(validate_dir,'with mask')
test_without_mask_dir = os.path.join(validate_dir,'without mask')
test_improper_mask_dir = os.path.join(validate_dir,'Improper mask')

In [14]:
print('Total training image for benign are :',len(os.listdir(test_with_mask_dir)))
print('Total training image for malignant are :',len(os.listdir(test_without_mask_dir)))
print('Total training image for malignant are :',len(os.listdir(test_improper_mask_dir)))


Total training image for benign are : 0
Total training image for malignant are : 1796
Total training image for malignant are : 2062


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_genrator = train_datagen.flow_from_directory(
    training_dir,
    target_size = (224,224),
    batch_size = 20,
    class_mode = 'binary'

)

test_genrator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (224,224),
    batch_size = 20,
    class_mode = 'binary'
)

In [ ]:

convo_base = ResNet50(weights='imagenet',
                   include_top=False,
                   input_shape=(224,224,3),
                   classes = 3
                   )

In [ ]:

model = models.Sequential()

In [ ]:
model.add(convo_base)

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(3,activation='softmax'))

In [ ]:
model.summary()


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              metrics = ['acc'],
               loss=tf.keras.losses.BinaryCrossentropy())


In [ ]:
check_point = tf.keras.callbacks.ModelCheckpoint(filepath="/content/transfer_l_mode.h5",
                                         monitor="val_acc",
                                         mode="max",
                                         save_best_only=True,)

In [ ]:
trained_model = model.fit(
    train_genrator,
    validation_data= test_genrator,
    epochs = 10,
    steps_per_epoch=131,
    #validation_steps=33
    callbacks = [check_point]
    )

In [ ]:
pd.DataFrame(trained_model.history).plot()

In [ ]:
model.evaluate(test_genrator)